In [1]:
import asyncio
import httpx

from langchain_core.messages import convert_to_messages, HumanMessage

from langgraph_sdk import get_client

In [2]:
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### Reject

In [3]:
# create a thread
thread = await client.threads.create()

# create to dos
user_input_1 = "Add a ToDo to follow-up with DI Repairs."
user_input_2 = "Add a ToDo to mount dresser to the wall."
config = {"configurable": {"user_id": "Test-Double-Texting"}}
graph_name = "task_maistro" 

run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)
try:
    await client.runs.create(
        thread["thread_id"],
        graph_name,
        input={"messages": [HumanMessage(content=user_input_2)]}, 
        config=config,
        multitask_strategy="reject",
    )
except httpx.HTTPStatusError as e:
    print("Failed to start concurrent run", e)

# wait until the original run completes
await client.runs.join(thread["thread_id"], run["run_id"])

# get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

Failed to start concurrent run Client error '409 Conflict' for url 'http://localhost:8123/threads/94a68c02-e01b-4f11-8541-de77d0bd0952/runs'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/409
================================ Human Message =================================

Add a ToDo to follow-up with DI Repairs.
================================== Ai Message ==================================

The task "Follow-up with DI Repairs" is already on your ToDo list. Is there anything else you would like to add or update?


### Enqueue

In [4]:
thread = await client.threads.create()

user_input_1 = "Send Erik his t-shirt gift this weekend."
user_input_2 = "Get cash and pay nanny for 2 weeks. Do this by Friday."
config = {"configurable": {"user_id": "Test-Double-Texting"}}
graph_name = "task_maistro" 

first_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="enqueue",
)

# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Send Erik his t-shirt gift this weekend.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_M1lBhZ4JJskOCTVZwV2pStNJ)
 Call ID: call_M1lBhZ4JJskOCTVZwV2pStNJ
  Args:
    update_type: todo
================================= Tool Message =================================

Document 10522f38-6f0f-46ad-9f0e-dd53bea82fe5 updated:
Plan: Update the deadline for sending Erik his t-shirt gift to this weekend, which is January 7, 2025.
Added content: 2025-01-07T23:59:59

Document 10522f38-6f0f-46ad-9f0e-dd53bea82fe5 updated:
Plan: Change the status of the task to 'in progress' as the user intends to send the gift this weekend.
Added content: in progress
================================== Ai Message ==================================

I've updated the deadline for sending Erik his t-shirt gift to this weekend, January 7, 2025. The task is now marked as "in

### Interrupt

In [5]:
thread = await client.threads.create()

user_input_1 = "Give me a summary of my ToDos due tomrrow."
user_input_2 = "Never mind, create a ToDo to Order Ham for Thanksgiving by next Friday."
config = {"configurable": {"user_id": "Test-Double-Texting"}}
graph_name = "task_maistro" 

interrupted_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

# wait for some of run 1 to complete so that we can see it in the thread 
await asyncio.sleep(1)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="interrupt",
)

# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Give me a summary of my ToDos due tomrrow.
================================ Human Message =================================

Never mind, create a ToDo to Order Ham for Thanksgiving by next Friday.
================================== Ai Message ==================================

You already have a task to "Order Ham for Thanksgiving" with a deadline of January 10, 2025. Would you like to update the deadline to next Friday instead?


In [6]:
# confirm that the first run was interrupted
print((await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"])

interrupted


### Rollback

In [7]:
thread = await client.threads.create()

user_input_1 = "Add a ToDo to call to make appointment at Yoga."
user_input_2 = "Actually, add a ToDo to drop by Yoga in person on Sunday."
config = {"configurable": {"user_id": "Test-Double-Texting"}}
graph_name = "task_maistro" 

rolled_back_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_1)]}, 
    config=config,
)

second_run = await client.runs.create(
    thread["thread_id"],
    graph_name,
    input={"messages": [HumanMessage(content=user_input_2)]}, 
    config=config,
    multitask_strategy="rollback",
)

# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

# get the state of the thread
state = await client.threads.get_state(thread["thread_id"])
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

Actually, add a ToDo to drop by Yoga in person on Sunday.
================================== Ai Message ==================================

It looks like the task "Drop by Yoga in person on Sunday" is already on your ToDo list. Is there anything else you'd like to add or modify?


In [8]:
# confirm that the original run was deleted
try:
    await client.runs.get(thread["thread_id"], rolled_back_run["run_id"])
except httpx.HTTPStatusError as _:
    print("Original run was correctly deleted")

Original run was correctly deleted
